# Extract data to a Spark DataFrame

In [7]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Example: Reading a CSV file
df = spark.read.option("sep", "\t").csv("data/en.openfoodfacts.org.products.csv.gz", header=True, inferSchema=True).cache()
# df.show()

In [8]:
df.columns

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'last_modified_by',
 'last_updated_t',
 'last_updated_datetime',
 'product_name',
 'abbreviated_product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'packaging_en',
 'packaging_text',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'origins',
 'origins_tags',
 'origins_en',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'ingredients_tags',
 'ingredients_analysis_tags',
 'allergens',
 'allergens_en',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'no_nutrition_data',
 'additives_n',
 'additives',
 'additives_tags',
 'additives_en',
 'nutriscore_score

# Generate a table in a CSV with statistical descriptions of the DataFrame

In [10]:
df.describe().toPandas().to_csv("describe_summary.csv", index=False)

# Take a sample of the DF

In [19]:
FRACTION_SIZE = 0.00001

In [20]:
columnsToSample = ['quantity', 'serving_size', 'serving_quantity', 'product_quantity']

In [21]:
samples_not_clean_df = df.select(columnsToSample).dropna(how='all').cache()

In [22]:
samples_df = samples_not_clean_df.sample(withReplacement=False, fraction=FRACTION_SIZE).cache()

In [23]:
samples_df.count()

20

In [24]:
samples_df.show()

+--------+------------------+----------------+----------------+
|quantity|      serving_size|serving_quantity|product_quantity|
+--------+------------------+----------------+----------------+
|    NULL| 1 TORTILLA (47 g)|            47.0|            NULL|
|  110 g.|              NULL|            NULL|           110.0|
|    4 oz|    1/2 cup (40 g)|            40.0|     113.3980925|
|    NULL|   22.5 g (22.5 g)|            22.5|            NULL|
|    NULL|     4 ONZ (113 g)|           113.0|            NULL|
|    NULL|  1 portion (28 g)|            28.0|            NULL|
|   150 g|              NULL|            NULL|           150.0|
|      25|              NULL|            NULL|            NULL|
|    NULL|              100g|           100.0|            NULL|
|   150 g|              NULL|            NULL|           150.0|
|    1pcs|              NULL|            NULL|            NULL|
|    NULL|1 portion (125 ml)|           125.0|            NULL|
|    500g|              NULL|           

# Drop null rows

In [86]:
dropped_rows = df.dropna(subset=['categories_tags'])

In [88]:
dropped_rows.show()

+-----------+--------------------+--------------+----------+-------------------+---------------+----------------------+----------------+--------------+---------------------+--------------------+------------------------+------------+--------+--------------------+------------------+------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+------------+----------+--------------------+-------------------------+-------------+----------------+----------------+----------------+----------------+------------------------+------+--------------------+---------------+-------------+---------+--------------+------------+--------------------+--------------------+-------------------------+-----------------+------------+------+-----------+---------+------------+----------------+-----------------+-----------+---------+--------------------+--------------------+----------------+----------------+----------+-------------------

# Extract unique strings in a column (ingredients analysis tags in this example)

In [4]:
def flatten_list(li):
    flat_list = []
    for row in li:
        flat_list += row
    return flat_list

def make_list_unique(li):
    return list(dict.fromkeys(li))

def split_string_list_elements(li, sep):
    return [x.split(sep) for x in li]

def column_to_list(col):
    return col.rdd.flatMap(lambda x: x).collect()

In [5]:
categories_list = make_list_unique(flatten_list(split_string_list_elements(column_to_list(df.select('ingredients_analysis_tags').dropna()), ",")))

In [6]:
categories_list

['en:palm-oil-free',
 'en:non-vegan',
 'en:vegetarian-status-unknown',
 'en:palm-oil-content-unknown',
 'en:vegan-status-unknown',
 'en:may-contain-palm-oil',
 'en:vegetarian',
 'en:vegan',
 'en:non-vegetarian',
 'en:palm-oil',
 'en:maybe-vegan',
 'en:maybe-vegetarian']

# Drop unecessary columns

In [26]:
kept_columns = ["code", "product_quantity", "energy-kcal_100g", "fat_100g", "saturated-fat_100g", "unsaturated-fat_100g", "monounsaturated-fat_100g", "polyunsaturated-fat_100g", "trans-fat_100g", 
                "carbohydrates_100g", "sugars_100g", "added-sugars_100g", "starch_100g", "fiber_100g", "proteins_100g", "allergens", "traces", "vitamin-a_100g", "vitamin-c_100g", "vitamin-d_100g",
                "vitamin-e_100g", "vitamin-k_100g", "vitamin-b1_100g", "vitamin-b2_100g", "vitamin-b6_100g", "vitamin-b9_100g", "vitamin-b12_100g", "calcium_100g",
                "iron_100g", "magnesium_100g", "potassium_100g", "zinc_100g", "food_groups_tags", "serving_size", "serving_quantity", "cholesterol_100g", "salt_100g", "glycemic-index_100g"]

In [27]:
df_kept_columns = df.select(kept_columns).cache()
df.unpersist()
df_kept_columns.columns

['code',
 'product_quantity',
 'energy-kcal_100g',
 'fat_100g',
 'saturated-fat_100g',
 'unsaturated-fat_100g',
 'monounsaturated-fat_100g',
 'polyunsaturated-fat_100g',
 'trans-fat_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'added-sugars_100g',
 'starch_100g',
 'fiber_100g',
 'proteins_100g',
 'allergens',
 'traces',
 'vitamin-a_100g',
 'vitamin-c_100g',
 'vitamin-d_100g',
 'vitamin-e_100g',
 'vitamin-k_100g',
 'vitamin-b1_100g',
 'vitamin-b2_100g',
 'vitamin-b6_100g',
 'vitamin-b9_100g',
 'vitamin-b12_100g',
 'calcium_100g',
 'iron_100g',
 'magnesium_100g',
 'potassium_100g',
 'zinc_100g',
 'food_groups_tags',
 'serving_size',
 'serving_quantity',
 'cholesterol_100g',
 'salt_100g',
 'glycemic-index_100g']

# Write DF to a database

## Get user's database credentials (not necessary)

In [ ]:
# properties = {}
# url = {}
# for text in ["database URL", "table name"]
#     url[text] = input("Enter " + text + ": ")


# for text in ["user", "password", "driver"]:
#     properties[text] = input("Enter " + text + ": ")

## Write to the database using JDBC Driver

In [5]:
properties = {
    "user": "user",
    "password": "userpassword",
    "driver": "com.mysql.cj.jdbc.Driver"
}

df_columns_kept.write.jdbc(url="jdbc:mysql://mysql:3306/pyspark_db", table="products", mode="append", properties=properties)

NameError: name 'df_filtered' is not defined

# GPT help